In [1]:
import xarray as xr
import numpy as np
import pandas as pd

import gitsummary
print(gitsummary.print_cwd_git_version())
import socket; print(f"Running on: {socket.getfqdn()}")

No git repository detected.
Running on: login24.chn


# MERRA2 - TECA BARD v1.0.1

This section uses TECA BARD to determine unambiguous AR presence over DC, IN.

In [2]:
""" Open the TECA ARTMIP files """

algorithm = "teca_bard_v1.0.1"
artmip_base =  "/global/cfs/cdirs/m1517/cascade/external_datasets/ARTMIP_NCAR_mirror/catalogues/tier1/"
file_glob = f"{artmip_base}/{algorithm}/MERRA2.ar_tag*.nc4"

teca_xr = xr.open_mfdataset(file_glob)
teca_xr

<xarray.Dataset>
Dimensions:              (time: 109568, lat: 361, lon: 576, dim_ar_count: 1024,
                          dim_parameter_table_row: 1024)
Coordinates:
  * lat                  (lat) float64 -90.0 -89.5 -89.0 ... 89.0 89.5 90.0
  * lon                  (lon) float64 -180.0 -179.4 -178.8 ... 178.8 179.4
  * time                 (time) datetime64[ns] 1980-01-01 ... 2017-06-30T21:0...
Dimensions without coordinates: dim_ar_count, dim_parameter_table_row
Data variables:
    ar_binary_tag        (time, lat, lon) int8 dask.array<chunksize=(2928, 361, 576), meta=np.ndarray>
    ar_count             (time, dim_ar_count) int32 dask.array<chunksize=(2928, 1024), meta=np.ndarray>
    ar_probability       (time, lat, lon) float32 dask.array<chunksize=(2928, 361, 576), meta=np.ndarray>
    parameter_table_row  (time, dim_parameter_table_row) int32 dask.array<chunksize=(2928, 1024), meta=np.ndarray>
Attributes:
    TECA_version:   4.1.0
    TECA_app_name:  teca_bayesian_ar_detect
    NCO:            netCDF Operators version 4.7.9 (Homepage = http://nco.sf....
    history:        Tue Jun 22 04:47:19 2021: ncatted -a long_name,time,c,c,t...

In [3]:
""" Pull out a point over DC """
blat, blon = 38.895, -77.036

teca_DC = teca_xr.sel(lat = blat, lon = blon, method = 'nearest')
teca_DC['indices'] = xr.DataArray(np.arange(0,len(teca_DC.time), dtype = int), dims = ('time',))
teca_DC

<xarray.Dataset>
Dimensions:              (time: 109568, dim_ar_count: 1024,
                          dim_parameter_table_row: 1024)
Coordinates:
  * time                 (time) datetime64[ns] 1980-01-01 ... 2017-06-30T21:0...
    lat                  float64 39.0
    lon                  float64 -76.88
Dimensions without coordinates: dim_ar_count, dim_parameter_table_row
Data variables:
    ar_binary_tag        (time) int8 dask.array<chunksize=(2928,), meta=np.ndarray>
    ar_count             (time, dim_ar_count) int32 dask.array<chunksize=(2928, 1024), meta=np.ndarray>
    ar_probability       (time) float32 dask.array<chunksize=(2928,), meta=np.ndarray>
    parameter_table_row  (time, dim_parameter_table_row) int32 dask.array<chunksize=(2928, 1024), meta=np.ndarray>
    indices              (time) int64 0 1 2 3 4 ... 109564 109565 109566 109567
Attributes:
    TECA_version:   4.1.0
    TECA_app_name:  teca_bayesian_ar_detect
    NCO:            netCDF Operators version 4.7.9 (Homepage = http://nco.sf....
    history:        Tue Jun 22 04:47:19 2021: ncatted -a long_name,time,c,c,t...

In [4]:
""" Find times with unambiguous AR conditions """

DC_ar = teca_DC.where(teca_DC['ar_probability'] == 1, drop = True)
DC_ar

<xarray.Dataset>
Dimensions:              (time: 1548, dim_ar_count: 1024,
                          dim_parameter_table_row: 1024)
Coordinates:
  * time                 (time) datetime64[ns] 1980-04-14T18:00:00 ... 2017-0...
    lat                  float64 39.0
    lon                  float64 -76.88
Dimensions without coordinates: dim_ar_count, dim_parameter_table_row
Data variables:
    ar_binary_tag        (time) float64 dask.array<chunksize=(43,), meta=np.ndarray>
    ar_count             (time, dim_ar_count) float64 dask.array<chunksize=(43, 1024), meta=np.ndarray>
    ar_probability       (time) float32 dask.array<chunksize=(43,), meta=np.ndarray>
    parameter_table_row  (time, dim_parameter_table_row) float64 dask.array<chunksize=(43, 1024), meta=np.ndarray>
    indices              (time) float64 dask.array<chunksize=(43,), meta=np.ndarray>
Attributes:
    TECA_version:   4.1.0
    TECA_app_name:  teca_bayesian_ar_detect
    NCO:            netCDF Operators version 4.7.9 (Homepage = http://nco.sf....
    history:        Tue Jun 22 04:47:19 2021: ncatted -a long_name,time,c,c,t...

In [5]:
""" Find only times in specific seasons """

DC_ar_season = {}
for season in ['DJF','MAM','JJA','SON']:
    DC_ar_season[season] = DC_ar.where(DC_ar['time.season'] == season, drop = True)
    # load the dataset
    DC_ar_season[season].load()
DC_ar_season['annual'] = DC_ar    

In [6]:
""" Write the indices to disk """

for season in DC_ar_season:
    indices = DC_ar_season[season]['indices'].values
    np.savetxt(f"merra2_index_files/dc_ars_{season}_{algorithm}.txt",indices,fmt = "%i")

# ERA5

This section converts the MERRA2 indices to indices in ERA5, assuming that the dataset is opened from 1979-01-01 00Z at hourly resolution, with time indices increasing linearly.

In [7]:
""" Create a time axis representing ERA5 """
e5_times = pd.DataFrame(dict(time = pd.date_range('1979-01-01 00:00:00', '2021-12-01 00:00:00', freq = '1H')))
e5_times = e5_times.reindex(e5_times['time'])#.to_xarray()
e5_times['time'] = e5_times.index
e5_times = e5_times.to_xarray()
e5_times['indices'] = xr.DataArray(np.arange(0,len(e5_times.time), dtype = int), dims = ('time',))
e5_times

<xarray.Dataset>
Dimensions:  (time: 376201)
Coordinates:
  * time     (time) datetime64[ns] 1979-01-01 1979-01-01T01:00:00 ... 2021-12-01
Data variables:
    indices  (time) int64 0 1 2 3 4 5 ... 376196 376197 376198 376199 376200

In [8]:
""" Convert to ERA5 indices and save. """
for season in DC_ar_season:
    e5_indices = e5_times.sel(time = DC_ar_season[season].time)['indices'].values
    np.savetxt(f"era5_index_files/dc_ars_{season}_era5_{algorithm}.txt",e5_indices,fmt = "%i")

Note: re-ran the cell above b/c file paths were incorrect.